In [14]:
from footprint import *
from test import generate_large_data_set
import numpy as np


## set conditions for data set generation
rtot = 0.02
sd_bp = 0.1
sd_height = 5
peaks_bp = np.arange(1,20,1)
footprinted_peaks_bp_and_kd = [[5,0.1],[6,0.5],[7,0.8]]
lfree_concs = np.random.random_integers(1,15,2) ## gets 30 random numbers in [1,15]

trace_list = generate_large_data_set.generate_data_set(rtot, peaks_bp, footprinted_peaks_bp_and_kd , lfree_concs, sd_bp, sd_height)

ref = trace_list[0]
traces = trace_list[1:]

print(len(trace_list),type(ref),len(traces))

cluster_peaks(ref,traces)

3 <class 'footprint.Trace'> 2


In [15]:
## original function


def calculate_deviance_for_all_peaks(ref, trace, weight_smaller=1,weight_bigger=1, weight_by_inverse_height=False, from_bp=0, to_bp=2000):
    '''
    Calculates the area between peaks that were identified as clustered in ref, 
    compared to trace. Allows for comparison of two traces only.
    
    
    Options weight_smaller, weight_bigger are used to consider only peaks from 
    trace which are smaller or bigger, respectively, than the reference peak.
    
    Option //BROKE relative_mode allows to calculate the deviation in relative terms to
    the reference peak.
    
    ## TODO: 
    from_bp .. to_bp 
    
    ## TODO: calculate deviance for trace_list --> saves computing time
    '''

    deviance_for_smaller_peaks = 0
    deviance_for_bigger_peaks = 0        
    num_smaller=0
    num_bigger=0
    
    for ref_peak,trace_peaks in give_all_clustered_peaks(ref,trace):
        ## WORKAROUND for single trace mode
        # if there are no peaks clustered to the ref_peak, they cannot be included --> continue with next pair
        if trace_peaks == []:
            print("WARNING: Not all peaks match -- omitting deviation for non-comparable peaks.")
            continue
        
        if from_bp < ref_peak.size_bp < to_bp:
            print("INFO: Trace contains peaks which are not included in deviation calculation.")

        ## allows to calculate deviance for one trace only
        trace_peak=trace_peaks[0]
        

        if weight_by_inverse_height == True: 
            ## this mode calculates deviation as percentage point, with different
            ## weights for each peak
            weight = 1/ref_peak.peak_height
        else:
            ## similarly weighted, the difference between the two trace functions
            ## ~ area / integral between traces is calculated
            weight = 1
        
        if trace_peak.peak_height <= ref_peak.peak_height:
            # deviation for smaller, i.e. potentially footprinted peaks
            deviance_for_smaller_peaks += abs((ref_peak.peak_height - trace_peak.peak_height)*weight)
            num_smaller +=1
        else:
            # deviation for bigger, i.e. potentially hypersensitive peaks
            deviance_for_bigger_peaks += abs((ref_peak.peak_height - trace_peak.peak_height)*weight)
            num_bigger +=1

    weighted_deviation = (weight_smaller*deviance_for_smaller_peaks + weight_bigger*deviance_for_bigger_peaks)

    ## possible further outputs:
    ## num_smaller
    ## num_bigger
    ## -> mean deviation per peak ~ normalizing

    return weighted_deviation
    

In [19]:
print(calculate_deviance_for_all_peaks(ref,trace_list[1],1,1,False,0,2000))

INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace 

In [29]:
storage = []

for trace in traces:
    for factor in np.linspace(start=0.5,stop=1.5,num=3):
        footprint.correct_peaks_with_factor(trace,factor)
        for weight_small, weight_big in [(0,1),(1,0)]:
            for relative in [True, False]:
                print(trace.file_name, factor,weight_small,weight_big,relative)
                index = " ".join([str(i) for i in [trace.file_name, factor,weight_small,weight_big,relative]])
                storage.append({index : calculate_deviance_for_all_peaks(ref,trace,weight_small,weight_big,relative)})
        footprint.correct_peaks_with_factor(trace,1/factor)

7 0.5 0 1 True
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculatio

In [30]:
print(storage)

[{'7 0.5 0 1 True': 0.0}, {'7 0.5 0 1 False': 0.0}, {'7 0.5 1 0 True': 16.977456386520849}, {'7 0.5 1 0 False': 51262.014652505255}, {'7 1.0 0 1 True': 0.0}, {'7 1.0 0 1 False': 0.0}, {'7 1.0 1 0 True': 14.954912773041693}, {'7 1.0 1 0 False': 45323.029305010532}, {'7 1.5 0 1 True': 0.0}, {'7 1.5 0 1 False': 0.0}, {'7 1.5 1 0 True': 12.932369159562544}, {'7 1.5 1 0 False': 39384.043957515787}, {'1 0.5 0 1 True': 0.0}, {'1 0.5 0 1 False': 0.0}, {'1 0.5 1 0 True': 10.059897940600013}, {'1 0.5 1 0 False': 30263.568315103767}, {'1 1.0 0 1 True': 0.023276765882073086}, {'1 1.0 0 1 False': 18.9896891187866}, {'1 1.0 1 0 True': 2.1430726470820991}, {'1 1.0 1 0 False': 6586.1263193263312}, {'1 1.5 0 1 True': 7.5178405039142575}, {'1 1.5 0 1 False': 21683.493435634031}, {'1 1.5 1 0 True': 1.6975343257142952}, {'1 1.5 1 0 False': 4554.1983809453468}]
